### Project 1

We need to design an dimplement a class that will be used to represent bank accounts.

We want the following functionality and characteristics:
- accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
- account holders have a **first** and **last** name
- accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
- **balances** need to be zero or higher, and should not be directly settable.
- but, **deposits** and **withdrawals** can be made (given sufficient funds)
    - if a withdrawal is attempted that would result in nagative funds, the transaction should be declined.
- a **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance.
- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)

For example, we may have an account with:
- account number `140568` 
- preferred time zone offset of -7 (MST) 
- an existing balance of `100.00`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

```D-140568-20190315145900-124```

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

Furthermore, if current interest rate is `0.5%`, and the account's balance is `1000.00`, then the result of calling the `deposit_interest` (or whatever name you choose) method, should result in a new transaction and a new balance of `1050.00`. Calling this method should also return a confirmation number.

For simplicty, just use floats, but be aware that for these types of situations you'll probably want to use `Decimal` objects instead of floats.

There are going to be many ways to design something like this, especially since I have not nailed down all the specific requirements, so you'll have to fill the gaps yourself and decide what other things you may want to implement (like is the account number going to be a mutable property, or "read-only" and so on).

See how many different ideas you can use from what we covered in the last section. 

My approach will end up creating two classes: a `TimeZone` class used to store the time zone name and offset definition (in hours and minutes), and a main class called `Account` that will have the following "public" interface:
- initializer with account number, first name, last name, optional preferred time zone, starting balance (defaults to 0)
- a first name property (read/write)
- a last name property (read/write)
- a full name property (computed, read-only)
- a balance property (read-only)
- an interest rate property (class level property)
- deposit, withdraw, pay_interest methods
- parse confirmation code

Class will have additional state and methods, but those will be used for implementation.

You should also remember to test your code! In the solutions I will introduce you to Python's `unittest` package. Even if you skip this project, at least review that video and/or notebook if you are unfamiliar with `unittest`.

In [ ]:
from datetime import datetime, timedelta, timezone
import re

In [ ]:
class Timer:
    
    tz = timezone.utc

    def __init__(self, offset=0) -> None:
        self.offset = offset
        self.tz = timezone(timedelta(hours=offset))

    @classmethod
    def get_current_dt(self):
        return self.tz
    
    @staticmethod
    def get_utc_time():
        return datetime.now(timezone.utc)
    
    def convert_utc_to_offset_timezone(self, timeobj):
        pass 


In [ ]:
class Result:    
    def __init__(self, confirmation_number) -> None:
        self.confirmation_number = confirmation_number
        self.account_number = self._validate_group(pattern=r'\w-(\d+)')
        self.transaction_code = self._validate_group(pattern=r'(\w)-\d+')
        self.transaction_id = self._validate_group(pattern=r'-(\d+)$')
        self.raw_timestring = self._validate_group(pattern=r'(\d+)-\d+$')

    def _validate_group(self, pattern):
        data = re.search(pattern, self.confirmation_number)
        if data:
            return data.group(1)
        else:
            raise AttributeError('Must be a valid confirmation number')

    @property 
    def timer(self):
        if not self.timer:
            return AttributeError('Timer not seted')
        return self.timer 

    @timer.setter
    def timer(self, timer):
        if not isinstance(timer, Timer):
            raise ValueError('Timer bust be a Timer object')
        time_utc_obj = datetime.strptime(self.raw_timestring, "%Y%m%d%H%M%S")
        self.time_utc = time_utc_obj.strftime('%Y-%m-%dT%h:%m:%S')
        time_offset_obj = time_utc_obj + timedelta(hours=timer.offset)
        self.time = time_offset_obj.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
class Account:

    _accounts = [] 
    _monthly_interest_rate = 0 
    _transaction_id = 0

    def __init__(self, account_number: int, balance: float, first_name: str, last_name: str, time_zone_offset: int) -> None:
        if not isinstance(account_number, int):
            raise TypeError('Account number must be integer')
        
        if not (isinstance(balance, float) or isinstance(balance, int)):
            raise TypeError('Balance must be a number.')
        
        if balance < 0:
            raise AttributeError('Balance must be zero or higher')
        
        self._add_new_account(account_number=account_number)
        self._account_number = account_number
        self._first_name = first_name
        self._last_name = last_name
        self._balance = balance
        self._time_zone_offset = time_zone_offset
        self.timer = Timer(time_zone_offset)

    @classmethod
    def _add_new_account(cls, account_number):
        if account_number in cls._accounts:
            raise AssertionError("Account number must be unique")
        cls._accounts.append(account_number)

    @classmethod
    def set_monthly_interest_rate(cls, value):
        if (isinstance(value, int) or isinstance(value, float)):
            cls._monthly_interest_rate = value 
        else:
            raise TypeError("Rate must be a number")

    @staticmethod
    def is_name_valid(value):
        if isinstance(value, str) and len(value) > 1:
            return True 
        return False 
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        if self.is_name_valid():
            self._first_name = value 
        else:
            raise AttributeError('Invalid name')
    
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        if self.is_name_valid():
            self._last_name = value 
        else:
            raise AttributeError('Invalid last name')
            
    @property
    def full_name(self):
        return ' '.join([self._first_name, self._last_name])
        
    @property
    def balance(self):
        return self._balance 
    
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def timezone_offset(self):
        return self._time_zone_offset
    
    @timezone_offset.setter
    def timezone_offset(self, value):
        self._time_zone_offset = value 

    def deposit(self, value):
        transaction_type = 'D'
        if not value > 0:
            raise ValueError("Deposits bust me greater than zero")
        self._balance += value 
        return self._generate_confirmation_number(transaction_type=transaction_type)

    def withdrawal(self, value):
        transaction_type = 'W' 
        
        if not value > 0:
            raise ValueError('Withdrawals must be greater than zero')
        
        if value <= self._balance:
            self._balance -= value
        else:
            transaction_type = 'X'
        
        return self._generate_confirmation_number(transaction_type=transaction_type)

    @classmethod
    def increase_transactions(cls):
        cls._transaction_id += 1 

    def _generate_confirmation_number(self, transaction_type):
        utc = ''.join(self.timer.get_utc_time().strftime('%Y%m%d%H%M%S'))
        confirmation_string = f"{transaction_type.upper()}-{self.account_number}-{utc}-{self._transaction_id}"
        if transaction_type != 'X':
            self.increase_transactions()
        return confirmation_string

    def parse_confirmation_code(self, confirmation_number):
        obj = Result(confirmation_number=confirmation_number)
        obj.timer = self.timer
        return obj 

    def account_interest_deposit(self):
        account_interest_rate = self._balance * self._monthly_interest_rate
        self._balance += account_interest_rate
        return self._generate_confirmation_number(transaction_type='I')

In [ ]:
person = Account(account_number=2, balance=100, first_name='Lorena', last_name='Miranda', time_zone_offset=-3)


In [ ]:
person.balance

In [ ]:
person.deposit(100)

In [ ]:
person.balance

In [ ]:
person.withdrawal(30)

In [ ]:
person.balance

In [ ]:
person.withdrawal(200)

In [ ]:
person.balance

In [ ]:
person.full_name

In [ ]:
person.__dict__

In [ ]:
result = person.parse_confirmation_code("W-2-20241105143023-1")
result.account_number, result.transaction_code, result.time_utc, result.time

In [ ]:
person.balance

In [ ]:
Account.set_monthly_interest_rate(value=0.05)

In [ ]:
person.account_interest_deposit()

In [ ]:
person.balance